# Tutorial

This tutorial explains how the simple standard name ontology can be used to enrich data.

Prerequisite:
- Knowledge about JSON format and [JSON-LD](https://json-ld.org/)

## Describing a StandardName

The JSON-LD content below describes a standard name "x_velocity", defined in a standard name table at (imaginary) URL "www.example.org/mytable".

We will write the content to a file and then read it with `rdflib` to show that everything works as expected:

In [1]:
import rdflib

The [context file](https://raw.githubusercontent.com/matthiasprobst/ssno/main/ssno_context.jsonld) is available in the GitHub repository. Simple include it via the "@import" key in the "@context" part.

In the following, we will describe the standard name "x_velocity". It has a canonical unit "m/s" (described by [qudt ontology](https://qudt.org/)) and a description. Every standard name either has a unique ID, hence a IRI, or is part of a *Standard Name Table* which is registered and accessible online. In the following, we use the latter, where "x_velocity" is part of the table, which can be found here: [https://doi.org/10.5281/zenodo.10428817](https://doi.org/10.5281/zenodo.10428817):

In [2]:
json_ld_content1 = """
{
  "@context": {
    "@import": "https://raw.githubusercontent.com/matthiasprobst/ssno/main/ssno_context.jsonld"
  },
  "@graph": [
    {
      "@id": "_:b0",
      "@type": "StandardName",
      "standard name": "x_velocity",
      "canonical units": "https://qudt.org/vocab/unit/M-PER-SEC",
      "description": "Velocity in x-axis direction.",
      "standard name table": "www.example.org/mytable"
    },
    {
      "@id": "https://doi.org/10.5281/zenodo.10428817",
      "@type": "StandardNameTable",
      "identifier": "https://doi.org/10.5281/zenodo.10428817"
    }
  ]
}
"""
print(json_ld_content1)


{
  "@context": {
    "@import": "https://raw.githubusercontent.com/matthiasprobst/ssno/main/ssno_context.jsonld"
  },
  "@graph": [
    {
      "@id": "_:b0",
      "@type": "StandardName",
      "standard name": "x_velocity",
      "canonical units": "https://qudt.org/vocab/unit/M-PER-SEC",
      "description": "Velocity in x-axis direction.",
      "standard name table": "www.example.org/mytable"
    },
    {
      "@id": "https://doi.org/10.5281/zenodo.10428817",
      "@type": "StandardNameTable",
      "identifier": "https://doi.org/10.5281/zenodo.10428817"
    }
  ]
}



## Write data to file

Writing the above to file is straight forward:

In [3]:
with open('standard_name.jsonld', 'w') as f:
    f.write(json_ld_content1)

## Exploring the data

JSON-LD data can be queried using the [SPARQL query language](https://www.w3.org/TR/sparql11-query/).

We will search for all standard names which include the substring "velocity". For the above example, this is straight forward, but for the sake of simplicity this works just fine:

In [4]:
g = rdflib.Graph()
g.parse('standard_name.jsonld', format='json-ld')

substring_name = 'velocity'

qres = g.query(f"""
PREFIX ssno: <https://matthiasprobst.github.io/ssno#>
PREFIX dcterms: <http://purl.org/dc/terms/>

SELECT ?standard_name ?description ?canonical_units
WHERE {{
    ?sn a ssno:StandardName .
    ?sn ssno:standardName ?standard_name .
    FILTER (CONTAINS(?standard_name, "{substring_name}"))
    ?sn ssno:description ?description .
    ?sn ssno:canonicalUnits ?canonical_units .
}}
""")

https://matthiasprobst.github.io/ssno#canonical units does not look like a valid URI, trying to serialize this will break.


Results:

In [5]:
[dict(name=str(row[0]), description=str(row[1]), canonical_units=str(row[2])) for row in qres]

[]

In [6]:
import pandas as pd

In [7]:
pd.DataFrame([dict(name=str(row[0]), description=str(row[1]), canonical_units=str(row[2])) for row in qres])

""


## Another example with multiple standard names

The origin of standard names lies in the [cf-conventions](http://cfconventions.org/). The table is provided as XML. 

In [8]:
g = rdflib.Graph()
g.parse(source='cf.json', format='json-ld')

substring_name = 'wind'

qres = g.query(f"""
PREFIX ssno: <https://matthiasprobst.github.io/ssno#>

SELECT ?standard_name ?description ?canonical_units
WHERE {{
    ?sn a ssno:StandardName .
    ?sn ssno:standardName ?standard_name .
    FILTER (CONTAINS(?standard_name, "{substring_name}"))
    ?sn ssno:description ?description .
    ?sn ssno:canonicalUnits ?canonical_units .
}}
""")

In [9]:
pd.DataFrame([dict(name=str(row[0]), description=str(row[1]), canonical_units=str(row[2])) for row in qres])

""


In [10]:
json_ld_content2 = """
{
  "@context": {
    "@import": "https://raw.githubusercontent.com/matthiasprobst/ssno/main/ssno_context.jsonld"
  },
  "@graph": [
    {
      "@id": "https://doi.org/10.5281/zenodo.10428817",
      "@type": "StandardNameTable",
      "doi": "https://doi.org/10.5281/zenodo.10428817",
      "has title": "My Table",
      "description": "My table description",
      "standard name": [
        {
          "@id": "_:b0",
          "@type": "StandardName",
          "standard_name": "x_velocity",
          "canonical_units": "m/s",
          "description": "Velocity in x-axis direction.",
          "standard name table": "https://doi.org/10.5281/zenodo.10428817"
        },
        {
          "@id": "_:b1",
          "@type": "StandardName",
          "standard_name": "y_velocity",
          "canonical_units": "m/s",
          "description": "Velocity in y-axis direction.",
          "standard name table": "https://doi.org/10.5281/zenodo.10428817"
        }
      ]
    }
  ]
}
"""

In [11]:
with open('standard_name_table.jsonld', 'w') as f:
    f.write(json_ld_content2)

In [12]:
g = rdflib.Graph()
g.parse('standard_name_table.jsonld', format='json-ld')

for s, p, o in sorted(g):
    print(s, '|', p, '|', o)

https://matthiasprobst.github.io/ssno#has title does not look like a valid URI, trying to serialize this will break.


b0 | http://www.w3.org/1999/02/22-rdf-syntax-ns#type | https://matthiasprobst.github.io/ssno#StandardName
b0 | https://matthiasprobst.github.io/ssno#canonical_units | m/s
b0 | https://matthiasprobst.github.io/ssno#description | Velocity in x-axis direction.
b0 | https://matthiasprobst.github.io/ssno#standardNameTable | https://doi.org/10.5281/zenodo.10428817
b0 | https://matthiasprobst.github.io/ssno#standard_name | x_velocity
b1 | http://www.w3.org/1999/02/22-rdf-syntax-ns#type | https://matthiasprobst.github.io/ssno#StandardName
b1 | https://matthiasprobst.github.io/ssno#canonical_units | m/s
b1 | https://matthiasprobst.github.io/ssno#description | Velocity in y-axis direction.
b1 | https://matthiasprobst.github.io/ssno#standardNameTable | https://doi.org/10.5281/zenodo.10428817
b1 | https://matthiasprobst.github.io/ssno#standard_name | y_velocity
https://doi.org/10.5281/zenodo.10428817 | http://www.w3.org/1999/02/22-rdf-syntax-ns#type | https://matthiasprobst.github.io/ssno#Standard